In [27]:
import os
%pip install -qU unstructured markdown pinecone

Note: you may need to restart the kernel to use updated packages.


In [30]:
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from dotenv import load_dotenv
from langchain_pinecone import Pinecone, PineconeVectorStore
from pinecone import ServerlessSpec, Pinecone as pinecone2
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
import os

with open("cs_response.md", "r", encoding="utf-8") as f:
    markdown_text = f.read()

load_dotenv()

embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3")
]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(markdown_text)

chunk_size = 500
chunk_overlap = 200
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(md_header_splits)

index_name = "cs-index"
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc = pinecone2(pinecone_api_key=pinecone_api_key)

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

database = PineconeVectorStore.from_documents(
    documents=[],
    embedding=embedding,
    index_name=index_name
)

['437457b9-afc6-42a9-b232-067f5b3125c9',
 '8393996e-90a8-415c-950e-760ace147a30',
 '78c3de1d-bfa3-458c-b651-fb097fe4fd21',
 'f2a7fd2c-740a-4165-9380-71e22514894d',
 '284a3626-cd49-40fa-aa2f-d90776e3662a',
 'bb61679e-49c9-47f6-b05e-d9d756a0b7c3',
 '5606cb41-8026-46f7-b20d-ff4acd9a9df7',
 'ee0e75c0-17e5-4982-9051-e6f49616cff8',
 '6e00ea15-6a2e-4461-bad6-282b5ed66cf9',
 'e42518c9-0239-4896-a4b5-251fb9b03758',
 'a10fabce-cdf4-439c-97e5-bb73d815dda5',
 '19d162d0-0448-45cb-b04b-1b945112f4fa',
 'a5ddb586-93e3-4d8f-979d-c763a50d03c8',
 '26de33c1-2023-41b7-9e36-1281ef3b4504',
 'abcfb83e-b181-46a7-942f-03c1c79ed86d',
 '22246dfe-42e3-4a93-a049-326b2a6979dd',
 'd9ff47a9-7788-4154-9dd0-712c56359e06',
 'aa2e8d23-2429-475b-aeec-a611335aa0ef',
 '9feb1b02-f2fd-461c-b624-b19ae7d6ee19',
 '6d7d9a82-73a3-445d-8088-12582392fc45',
 'ee245a38-dc5f-4126-ab76-f866759a0841',
 '1bfcea5a-ffd4-418c-b5ac-3c8900caa845',
 '10f7c24b-27b8-4129-970b-4f1e4a4f06c8',
 '7e35a460-43ce-404c-b56d-2d5559d55d17',
 'dfebb630-0c50-

In [82]:
from langchain_upstage import ChatUpstage
llm = ChatUpstage(temperature=0)

query = '멀티 프로세스란 무엇인가요?'
retrievd_docs = database.similarity_search(query, k = 3)
print(retrievd_docs)

[Document(id='5606cb41-8026-46f7-b20d-ff4acd9a9df7', metadata={'Header 1': '기술 면접 대비', 'Header 2': '운영체제', 'Header 3': 'Multi Process'}, page_content='멀티 프로세스란 2개 이상의 프로세스가 동시에 실행되는 것을 말합니다.  \n2개 이상의 프로세스가 동시에 실행되는 데, 이때 프로세스들은 CPU와 메모리를 공유하게 됩니다. 메모리의 경우에는 여러 프로세스들이 각자의 메모리 영역을 차지하여 동시에 적재됩니다.  \n반면 하나의 CPU는 매 순간 하나의 프로세스만 연산할 수 있습니다. 하지만 CPU의 처리 속도가 워낙 빨라서 짧은 시간동안 여러 프로세스들이 CPU에서 번갈아 실행되기 때문에 사용자 입장에서는 여러 프로그램이 동시에 실행되는 것 처럼 보입니다. 이처럼 CPU의 작업시간을 여러 프로세스들이 조금씩 나누어 쓰는 시스템을 시분할 시스템(Time sharing System) 이라고 합니다.'), Document(id='284a3626-cd49-40fa-aa2f-d90776e3662a', metadata={'Header 1': '기술 면접 대비', 'Header 2': '운영체제', 'Header 3': 'CPU의 연산'}, page_content='- 멀티 프로세스(Multi Process) (★★★★)  \n멀티 프로세스란 2개 이상의 프로세스가 동시에 실행되는 것을 말합니다.  \n동시에 라는 말은 동시성(Concurrency)와 병렬성(parallelism) 두가지를 의미합니다.  \n**동시성**은 CPU 코어가 한개(싱글코어)일때, 여러 프로세스를 짧은 시간동안 번갈아 가면서 연산을 하게 되는 시분할 시스템으로 실행되는 것입니다.  \n(중요함, 면접에서 자주 나옴)  \n시분할 시스템 : CPU의 작업시간을 여러 process들이 조금씩 나누어 쓰는 시스템을 시분할 시스템(time sharing system)이라고 부릅니다.  

In [100]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
당신은 컴퓨터 공학 교수입니다.
제공된 <context> 를 바탕으로 다음 지침에 따라 답변하세요.

[예시]
- 질문이 문맥에 있는 경우:
답변: [내용]
코멘트 : [쉬운 설명]

- 질문이 문맥에 없는 경우:
존재하지 않는 내용입니다.

[지침]
1. 질문이 문맥과 관련 없으면 오직 "존재하지 않는 내용입니다."만 출력하세요.
2. 질문이 문맥에 있는 경우, 문맥에 있는 내용에 더불어 5살도 이해할 수 있는 쉬운 설명을 "코멘트 : " 를 시작으로 포함하여 답변해주세요.

<context>
{context}
</context>

질문: {input}

""")

combine_docs_chain = create_stuff_documents_chain(llm, prompt)

retriever = database.as_retriever()
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [111]:
query = '쓰레드가 뭐야?'
response = rag_chain.invoke({"input": query})
print(response["answer"])

답변: 쓰레드는 한 프로세스 내에서 실행되는 독립적인 실행 흐름입니다. 각 쓰레드는 속해있는 프로세스의 Stack 메모리를 제외한 나머지 Memory 영역을 공유할 수 있습니다.

코멘트 : 쓰레드는 하나의 프로세스 안에서 돌아가는 작은 작업 단위라고 생각하면 돼요. 하나의 프로세스 안에 여러 개의 쓰레드가 있을 수 있고, 각 쓰레드는 자신의 일을 하면서 프로세스가 사용하는 메모리의 일부를 공유할 수 있어요.
